## 기본설정 및 함수정의

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
sejong = [36.51430420729354, 127.29034973889941 ]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 세종시 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [34]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_002/_census_data_2023_bnd_dong_bnd_dong_29010_2023_2023"
sejong_gdf = gpd.read_file(shapefile_path)
sejong_gdf = sejong_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환
# 조건에 따라 'newtown' 열 생성 및 값 지정
sejong_gdf['newtown'] = sejong_gdf['ADM_NM'].apply(lambda x: '신도시' if x.endswith('동') else '그외지역')

#### 격자(매핑용)

In [36]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/4.세종시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_map = KeplerGl(height=1000, width=1500)
sejong_map.add_data(data=sejong_gdf, name="세종시 행정동 데이터")
sejong_map.add_data(data=grid_map_df, name="세종시 격자 데이터")

# 맵 출력 및 상세설정
sejong_map

In [40]:
# 맵 저장
sejong_map.save_to_html(file_name="visualization/세종시 현황/세종시 기본정보 map.html")

Map saved to visualization/세종시 현황/세종시 기본정보 map.html!


#### 격자인구통계

In [84]:
grid_dic = dict(zip(grid_map_df['properties.gid'], grid_map_df['geometry']))
grid_pol = grid_map_df['properties.gid'].unique()

# 연도 범위 설정 & 파일 경로 템플릿 설정
years = range(2020, 2023)
file_template = "SBJ_2406_002/_census_reqdoc_1721628002910/{}년_인구_다바_100M.txt"
# 각 연도에 대해 파일 로드 및 데이터프레임 변환
dfs = []
columns_order = ['year', 'gid', 'option', 'pop']
for year in years:
    file_path = file_template.format(year)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, delimiter='^', encoding='utf-8', header=None, names=columns_order)
        df.drop(columns='option', inplace=True)
        df = df[df['gid'].isin(grid_pol)]
        df['geometry'] = df['gid'].map(grid_dic)
        dfs.append(df)
# 데이터프레임을 하나로 병합
grid_pop_df = pd.concat(dfs, ignore_index=True)
grid_pop_df = gpd.GeoDataFrame(grid_pop_df, geometry='geometry')

In [118]:
# Plotly를 사용하여 히스토그램 시각화
fig = px.histogram(grid_pop_df, x='pop', nbins=10, labels={'pop': 'Population'})
# 그림 크기 조절
fig.update_layout(
    width=800,  # 원하는 너비로 설정
    height=600  # 원하는 높이로 설정
)
fig.show()

In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_gridpop_map = KeplerGl(height=1000, width=1500)
sejong_gridpop_map.add_data(data=grid_pop_df, name="세종시 격자거주인구 데이터")
sejong_gridpop_map.add_data(data=sejong_gdf, name="세종시 행정구역 데이터")

# 맵 출력 및 상세설정
sejong_gridpop_map

In [91]:
# 맵 저장
sejong_gridpop_map.save_to_html(file_name="visualization/세종시 현황/세종시 격자거주인구 map.html")

Map saved to visualization/세종시 현황/세종시 격자거주인구 map.html!


#### 행정구역별 인구(격자거주인구 활용)

In [102]:
# 공간 조인 수행
joined_df = gpd.sjoin(grid_pop_df, sejong_gdf, how='inner', op='intersects')

# 연도별로 행정구역의 인구수를 합산
sejong_pop_gdf = joined_df.groupby(['year', 'ADM_NM']).agg({'pop': 'sum'}).reset_index()
sejong_pop_gdf = pd.merge(sejong_pop_gdf, sejong_gdf[['ADM_NM', 'geometry']], on='ADM_NM')
sejong_pop_gdf = gpd.GeoDataFrame(sejong_pop_gdf, geometry='geometry')

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.

C:\Users\HJK\AppData\Local\Temp\ipykernel_37072\2332084221.py:2: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326




In [129]:
sejong_gdf[sejong_gdf['newtown'] == '신도시']['ADM_NM'].tolist()

,BASE_DATE,ADM_NM,ADM_CD,geometry,newtown
9,20230701,반곡동,29010513,"MULTIPOLYGON (((127.34757 36.51357, 127.34758 ...",신도시
11,20230701,종촌동,29010560,"POLYGON ((127.24233 36.50869, 127.24234 36.508...",신도시
12,20230701,아름동,29010590,"POLYGON ((127.25182 36.52672, 127.25193 36.526...",신도시
13,20230701,고운동,29010600,"POLYGON ((127.23853 36.53540, 127.23895 36.535...",신도시
14,20230701,한솔동,29010610,"POLYGON ((127.26243 36.47868, 127.26181 36.478...",신도시
15,20230701,대평동,29010640,"POLYGON ((127.28217 36.46873, 127.28215 36.468...",신도시
16,20230701,보람동,29010660,"POLYGON ((127.29381 36.48496, 127.29381 36.484...",신도시
17,20230701,나성동,29010522,"POLYGON ((127.25883 36.48370, 127.25885 36.483...",신도시
18,20230701,다정동,29010680,"POLYGON ((127.23703 36.49965, 127.23710 36.499...",신도시
19,20230701,어진동,29010515,"POLYGON ((127.25357 36.50688, 127.25357 36.506...",신도시


In [121]:
sejong_pop_gdf

,year,ADM_NM,pop,geometry
0,2020,고운동,70030,"POLYGON ((127.23853 36.53540, 127.23895 36.535..."
1,2021,고운동,69528,"POLYGON ((127.23853 36.53540, 127.23895 36.535..."
2,2022,고운동,70194,"POLYGON ((127.23853 36.53540, 127.23895 36.535..."
3,2020,금남면,19876,"POLYGON ((127.36393 36.51578, 127.36424 36.513..."
4,2021,금남면,19745,"POLYGON ((127.36393 36.51578, 127.36424 36.513..."
...,...,...,...,...
67,2021,한솔동,40323,"POLYGON ((127.26243 36.47868, 127.26181 36.478..."
68,2022,한솔동,39181,"POLYGON ((127.26243 36.47868, 127.26181 36.478..."
69,2020,해밀동,8406,"POLYGON ((127.26824 36.54367, 127.26841 36.543..."
70,2021,해밀동,16491,"POLYGON ((127.26824 36.54367, 127.26841 36.543..."


In [131]:
# 연도별로 행정구역의 인구수 합산
pop_trend_df = sejong_pop_gdf.groupby(['year', 'ADM_NM']).agg({'pop': 'sum'}).reset_index()
# 각 행정구역의 총 인구수 계산
total_pop_df = pop_trend_df.groupby('ADM_NM')['pop'].sum().reset_index()

# 총 인구수를 기준으로 행정구역 정렬
total_pop_df = total_pop_df.sort_values(by='pop', ascending=False)

# 각 행정구역별로 데이터를 분리하여 선과 마커를 동시에 표시하는 그래프 생성
fig = go.Figure()

for adm_nm in total_pop_df['ADM_NM'].unique():
    df = pop_trend_df[pop_trend_df['ADM_NM'] == adm_nm]
    fig.add_trace(go.Scatter(x=df['year'], y=df['pop'], mode='lines+markers', name=adm_nm))

# 그래프 레이아웃 설정
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Population',
    width=800,
    height=600
)

# 그래프 시각화
fig.show()

In [138]:
newtown_lst = sejong_gdf[sejong_gdf['newtown'] == '신도시']['ADM_NM'].tolist()
# 연도별로 행정구역의 인구수 합산
pop_trend_df = sejong_pop_gdf.groupby(['year', 'ADM_NM']).agg({'pop': 'sum'}).reset_index()
pop_trend_df = pop_trend_df[pop_trend_df['ADM_NM'].isin(newtown_lst)]
# 각 행정구역의 총 인구수 계산
total_pop_df = pop_trend_df.groupby('ADM_NM')['pop'].sum().reset_index()

# 총 인구수를 기준으로 행정구역 정렬
total_pop_df = total_pop_df.sort_values(by='pop', ascending=False)

# 각 행정구역별로 데이터를 분리하여 선과 마커를 동시에 표시하는 그래프 생성
fig = go.Figure()

for adm_nm in total_pop_df['ADM_NM'].unique():
    df = pop_trend_df[pop_trend_df['ADM_NM'] == adm_nm]
    fig.add_trace(go.Scatter(x=df['year'], y=df['pop'], mode='lines+markers', name=adm_nm))

# 그래프 레이아웃 설정
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Population',
    width=800,
    height=600
)

# 그래프 시각화
fig.show()

In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_pop_map = KeplerGl(height=1000, width=1500)
sejong_pop_map.add_data(data=sejong_pop_gdf, name="세종시 거주인구 데이터")

# 맵 출력 및 상세설정
sejong_pop_map

In [106]:
# 맵 저장
sejong_pop_map.save_to_html(file_name="visualization/세종시 현황/세종시 거주인구 map.html")

Map saved to visualization/세종시 현황/세종시 거주인구 map.html!


#### 연속지적도

In [ ]:
cadastral_dic = {'전' : '밭',
                '답' : '논',
                '과' : '과수원',
                '목' : '목장',
                '임' : '임야',
                '광' : '광천지',
                '염' : '염전',
                '대' : '대지',
                '장' : '공장용지',
                '학' : '학교용지',
                '차' : '주차장',
                '주' : '주유소용지',
                '창' : '창고용지',
                '도' : '도로',
                '철' : '철도용지',
                '제' : '제방',
                '천' : '하천',
                '구' : '도랑',
                '유' : '유지',
                '양' : '양어장',
                '수' : '수도용지',
                '공' : '공원용지',
                '체' : '체육용지',
                '원' : '유원지',
                '종' : '종교용지',
                '사' : '사적지',
                '묘' : '묘지',
                '잡' : '잡종지',
                '기타' : '기타'}

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/8.세종시_연속지적도.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
cadastral_map_df = pd.json_normalize(geojson_data['features'])
cadastral_map_df['geometry'] = cadastral_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
cadastral_map_df.drop(columns="geometry.coordinates", inplace=True)
# cadastral_map_df 데이터프레임을 GeoDataFrame으로 변환
cadastral_map_df = gpd.GeoDataFrame(cadastral_map_df, geometry='geometry')
# 정규식을 사용하여 'properties.JIBUN'을 지번과 용도로 분리
# 지번은 숫자 또는 숫자-숫자 형식, 용도는 공백 뒤에 오는 한글 문자로 가정
cadastral_map_df['JIBUN'] = cadastral_map_df['properties.JIBUN'].str.extract(r'(\d+-?\d*)')
cadastral_map_df['USAGE'] = cadastral_map_df['properties.JIBUN'].apply(lambda x:x[-1])
# USAGE가 빈 문자열이거나 숫자로만 되어 있는 경우 '기타'로 설정
cadastral_map_df['USAGE'] = cadastral_map_df['USAGE'].apply(lambda x: '기타' if x == '' or x.isdigit() else x)
cadastral_map_df['USAGE'] = cadastral_map_df['USAGE'].apply(lambda x: cadastral_dic.get(x, x))
cadastral_map_df = cadastral_map_df[['JIBUN', 'USAGE', 'geometry']]
cadastral_map_df

,JIBUN,USAGE,geometry
0,59-14,임야,"POLYGON ((127.26892 36.48192, 127.26901 36.481..."
1,223-1,논,"POLYGON ((127.30726 36.46622, 127.30716 36.466..."
2,295-4,밭,"POLYGON ((127.18762 36.69594, 127.18766 36.695..."
3,63,임야,"POLYGON ((127.31121 36.47124, 127.31127 36.471..."
4,296,밭,"POLYGON ((127.18777 36.69513, 127.18783 36.695..."
...,...,...,...
205429,450-10,밭,"POLYGON ((127.27924 36.54751, 127.27915 36.547..."
205430,450-7,밭,"POLYGON ((127.27924 36.54751, 127.27926 36.547..."
205431,450-11,밭,"POLYGON ((127.27953 36.54758, 127.27926 36.547..."
205432,451-3,밭,"POLYGON ((127.27953 36.54758, 127.27965 36.547..."


In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_cadastral_map = KeplerGl(height=1000, width=1500)
sejong_cadastral_map.add_data(data=sejong_gdf, name="세종시 행정동 데이터")
sejong_cadastral_map.add_data(data=cadastral_map_df, name="세종시 연속지적도")

# 맵 출력 및 상세설정
sejong_cadastral_map

In [ ]:
# 맵 저장
sejong_cadastral_map.save_to_html(file_name="visualization/세종시 현황/세종시 연속지적도 map.html")

Map saved to visualization/세종시 현황/세종시 연속지적도 map.html!


#### 세대별 유동인구

In [189]:
# 유동인구 df 로드
floating_population_df = pd.read_csv('SBJ_2406_001/27.영등포_성연령별_유동인구.csv')
floating_population_df['STD_YM'] = floating_population_df['STD_YM'].astype(str)
floating_population_df['pop'] = floating_population_df.iloc[:, 1:-3].sum(axis=1).copy()
floating_population_df['STD_YM'] = floating_population_df['STD_YM'].apply(lambda x:x[:4])

# 연령대별로 성별 통합
floating_population_df['10대'] = floating_population_df['w_10g_pop'] + floating_population_df['m_10g_pop']
floating_population_df['20대'] = floating_population_df['w_20g_pop'] + floating_population_df['m_20g_pop']
floating_population_df['30대'] = floating_population_df['w_30g_pop'] + floating_population_df['m_30g_pop']
floating_population_df['40대'] = floating_population_df['w_40g_pop'] + floating_population_df['m_40g_pop']
floating_population_df['50대'] = floating_population_df['w_50g_pop'] + floating_population_df['m_50g_pop']
floating_population_df['60대'] = floating_population_df['w_60g_pop'] + floating_population_df['m_60g_pop']
col_lst = list(floating_population_df.columns[-6:]) + list(floating_population_df.columns[-7:-6]) + list(floating_population_df.columns[-9:-7])
floating_population_df = floating_population_df[col_lst]
floating_population_df = floating_population_df.groupby(['lon', 'lat']).mean().reset_index()
floating_population_df

,lon,lat,10대,20대,30대,40대,50대,60대,pop
0,126.878273,37.552921,0.323333,0.811111,1.671389,2.402778,2.266389,2.240556,9.070000
1,126.878276,37.552470,0.623056,1.660556,3.490278,4.966111,4.565000,4.723611,18.598889
2,126.878280,37.552020,0.636389,1.625556,3.372500,4.701667,4.421667,4.536111,18.021944
3,126.878284,37.551569,1.211944,2.606111,5.427222,7.280556,6.731667,7.047222,28.323333
4,126.878549,37.519573,2.923333,5.701667,12.430278,18.447500,19.311944,19.977222,72.642500
...,...,...,...,...,...,...,...,...,...
9517,126.949843,37.519476,0.405625,1.095312,1.339688,1.475625,1.505000,2.110313,7.067813
9518,126.949846,37.519025,0.252500,1.178333,1.551111,1.599167,1.438056,1.603333,7.030000
9519,126.949849,37.518574,0.123611,0.507778,0.873889,1.025000,0.940833,0.923611,4.113333
9520,126.949853,37.518124,0.112857,0.522000,0.662000,0.713143,0.655143,0.710000,3.111429


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_floatingpop_map = KeplerGl(height=1000, width=1500)
ydp_floatingpop_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_floatingpop_map.add_data(data=floating_population_df, name="영등포구 세대별 유동인구 데이터")
ydp_floatingpop_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_floatingpop_map

In [192]:
# 맵 저장
ydp_floatingpop_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 세대별 유동인구 map.html")

Map saved to visualization/영등포구 현황/영등포구 세대별 유동인구 map.html!


#### 시간대별 유동인구

In [ ]:
# 유동인구 df 로드
floating_population_df_T = pd.read_csv('SBJ_2406_001/28.영등포_시간대별_유동인구.csv')
floating_population_df_T['STD_YM'] = floating_population_df_T['STD_YM'].astype(str)
floating_population_df_T['STD_YM'] = floating_population_df_T['STD_YM'].apply(lambda x:x[:4])

# 열 이름 변경 과정
columns_to_rename = floating_population_df_T.columns[1:-2] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    numeric_part = column_name.split('_')[-1] # 숫자 부분 추출
    new_column_name = f"2020-01-01 {numeric_part}:00" # '시'를 붙여 새로운 열 이름 생성
    new_column_names.append(new_column_name) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_T.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_T =floating_population_df_T.iloc[:,1:]
floating_population_df_T = floating_population_df_T.groupby(['lon', 'lat']).mean().reset_index()

# 시간대별 유동인구 정보를 'timestamp' 형식으로 변환
def melt_time_columns(df):
    id_vars = ['lon', 'lat']
    value_vars = [col for col in df.columns if col not in id_vars]
    df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='time', value_name='population')
    return df_melted

floating_population_df_T = melt_time_columns(floating_population_df_T)
floating_population_df_T

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_floatingpop_T_map = KeplerGl(height=1000, width=1500)
ydp_floatingpop_T_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")
ydp_floatingpop_T_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_floatingpop_T_map.add_data(data=floating_population_df_T, name="영등포구 시간대별 유동인구 데이터")

# 맵 출력 및 상세설정
ydp_floatingpop_T_map

In [219]:
# 맵 저장
ydp_floatingpop_T_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 시간대별 유동인구 map.html")

Map saved to visualization/영등포구 현황/영등포구 시간대별 유동인구 map.html!


#### 요일별 유동인구

In [14]:
floating_population_df_D = pd.read_csv('SBJ_2406_001/29.영등포_요일별_유동인구.csv')

floating_population_df_D['STD_YM'] = floating_population_df_D['STD_YM'].astype(str)
floating_population_df_D['STD_YM'] = floating_population_df_D['STD_YM'].apply(lambda x:x[:4])

# 열 이름 변경 과정
columns_to_rename = floating_population_df_D.columns[1:-2] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    day_part = column_name.split('_')[0] # 숫자 부분 추출
    new_column_names.append(day_part) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_D.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_D =floating_population_df_D.iloc[:,1:]
floating_population_df_D = floating_population_df_D.groupby(['lon', 'lat']).mean().reset_index()

# 평일 및 주말 평균 계산
def calculate_weekday_weekend_means(df):
    # 평일 평균 = 1
    df['1'] = df[['mon', 'tue', 'wed', 'thu', 'fri']].mean(axis=1)
    # 주말 평균 = 2
    df['2'] = df[['sat', 'sun']].mean(axis=1)
    return df[['lon', 'lat', '1', '2']]

floating_population_df_D = calculate_weekday_weekend_means(floating_population_df_D)

# long 형식으로 변환
floating_population_df_D = floating_population_df_D.melt(
    id_vars=['lon', 'lat'],
    value_vars=['1', '2'],
    var_name='day_type',
    value_name='population'
)

floating_population_df_D

,lon,lat,day_type,population
0,126.878273,37.552921,1,12.285167
1,126.878276,37.552470,1,25.560667
2,126.878280,37.552020,1,24.682944
3,126.878284,37.551569,1,34.131389
4,126.878549,37.519573,1,84.020222
...,...,...,...,...
19237,126.949843,37.519476,2,7.803971
19238,126.949846,37.519025,2,6.366528
19239,126.949849,37.518574,2,3.624583
19240,126.949853,37.518124,2,2.748000


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_floatingpop_D_map = KeplerGl(height=1000, width=1500)
ydp_floatingpop_D_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")
ydp_floatingpop_D_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_floatingpop_D_map.add_data(data=floating_population_df_D, name="영등포구 평일 및 주말 유동인구 데이터")

# 맵 출력 및 상세설정
ydp_floatingpop_D_map

In [16]:
# 맵 저장
ydp_floatingpop_D_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 주중 및 주말 유동인구 map.html")

Map saved to visualization/영등포구 현황/영등포구 주중 및 주말 유동인구 map.html!
